In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('data/Traffic_Crashes_People.csv', low_memory=False)

In [3]:
df.drop(['PERSON_ID', 'PERSON_TYPE', 'RD_NO', 'VEHICLE_ID', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 
         'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED', 'EJECTION', 
         'INJURY_CLASSIFICATION', 'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 
         'PEDPEDAL_LOCATION', 'BAC_RESULT VALUE', 'CELL_PHONE_USE'], axis=1, inplace=True)
df.columns

Index(['CRASH_RECORD_ID', 'CRASH_DATE', 'AGE', 'DRIVER_ACTION',
       'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT'],
      dtype='object')

In [6]:
#delete age nulls, set to drivers above 16 only
df.dropna(subset=['AGE'], inplace=True)
# df['AGE'].isna().sum()
df = df[df['AGE']>=16]

In [17]:
#drop rest of the null values
df.dropna(subset=['DRIVER_ACTION'], inplace=True)
df.dropna(subset=['DRIVER_VISION'], inplace=True)
df.dropna(subset=['BAC_RESULT'], inplace=True)
df.dropna(subset=['PHYSICAL_CONDITION'], inplace=True)

In [18]:
df.isna().sum()

CRASH_RECORD_ID       0
CRASH_DATE            0
AGE                   0
DRIVER_ACTION         0
DRIVER_VISION         0
PHYSICAL_CONDITION    0
BAC_RESULT            0
dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 708237 entries, 0 to 1224557
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   CRASH_RECORD_ID     708237 non-null  object 
 1   CRASH_DATE          708237 non-null  object 
 2   AGE                 708237 non-null  float64
 3   DRIVER_ACTION       708237 non-null  object 
 4   DRIVER_VISION       708237 non-null  object 
 5   PHYSICAL_CONDITION  708237 non-null  object 
 6   BAC_RESULT          708237 non-null  object 
dtypes: float64(1), object(6)
memory usage: 43.2+ MB


In [20]:
df['DRIVER_ACTION'].value_counts()

NONE                                 320325
UNKNOWN                              118669
OTHER                                 66729
FAILED TO YIELD                       65164
FOLLOWED TOO CLOSELY                  43065
IMPROPER BACKING                      18725
IMPROPER TURN                         18261
IMPROPER LANE CHANGE                  16548
IMPROPER PASSING                      11297
TOO FAST FOR CONDITIONS               10516
DISREGARDED CONTROL DEVICES           10368
IMPROPER PARKING                       2235
WRONG WAY/SIDE                         2153
CELL PHONE USE OTHER THAN TEXTING      1130
OVERCORRECTED                           952
EVADING POLICE VEHICLE                  910
EMERGENCY VEHICLE ON CALL               770
TEXTING                                 292
STOPPED SCHOOL BUS                       93
LICENSE RESTRICTIONS                     35
Name: DRIVER_ACTION, dtype: int64

In [21]:
df['DRIVER_VISION'].value_counts()

NOT OBSCURED              482888
UNKNOWN                   202536
OTHER                       9034
MOVING VEHICLES             5566
PARKED VEHICLES             3368
WINDSHIELD (WATER/ICE)      2674
BLINDED - SUNLIGHT          1103
TREES, PLANTS                431
BUILDINGS                    333
BLINDED - HEADLIGHTS          83
HILLCREST                     74
BLOWING MATERIALS             67
EMBANKMENT                    59
SIGNBOARD                     21
Name: DRIVER_VISION, dtype: int64

In [22]:
df['PHYSICAL_CONDITION'].value_counts()

NORMAL                          599773
UNKNOWN                          91971
IMPAIRED - ALCOHOL                4252
REMOVED BY EMS                    3345
FATIGUED/ASLEEP                   2397
OTHER                             2208
EMOTIONAL                         1949
ILLNESS/FAINTED                    875
HAD BEEN DRINKING                  632
IMPAIRED - DRUGS                   510
IMPAIRED - ALCOHOL AND DRUGS       213
MEDICATED                          112
Name: PHYSICAL_CONDITION, dtype: int64

In [23]:
df['BAC_RESULT'].value_counts()

TEST NOT OFFERED                   694723
TEST REFUSED                         9508
TEST PERFORMED, RESULTS UNKNOWN      2211
TEST TAKEN                           1795
Name: BAC_RESULT, dtype: int64